In [1]:
# Install missing packages
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, make_scorer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re
from collections import Counter
import spacy
import nltk




In [2]:
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
        
# path = '/kaggle/input/mbd2023-food-safety-controls/'

In [3]:
import os

# get the current working directory
current_working_directory = os.getcwd()

# print output to the console
print(current_working_directory)

C:\Users\mperan\OneDrive - IESEG\Cours M2\DD\S2\SML\Group project


In [4]:
# Read data 

train = pd.read_csv("food_safety_train.csv", encoding='latin1')
test = pd.read_csv("food_safety_test.csv", encoding='latin1')                   

### 1. Import and prepare data

#### 1.1. Train data

In [5]:
# # Read data
# train = pd.read_csv(path + 'food_safety_train.csv', encoding='latin1')
# test = pd.read_csv(path + 'food_safety_test.csv', encoding='latin1')

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22671 entries, 0 to 22670
Data columns (total 14 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   trustii_id                          22671 non-null  int64 
 1   APP_Libelle_etablissement           22670 non-null  object
 2   SIRET                               22671 non-null  object
 3   Adresse_2_UA                        22359 non-null  object
 4   Code_postal                         22671 non-null  object
 5   Libelle_commune                     22671 non-null  object
 6   Numero_inspection                   22671 non-null  object
 7   Date_inspection                     22671 non-null  object
 8   APP_Libelle_activite_etablissement  22671 non-null  object
 9   Agrement                            5823 non-null   object
 10  geores                              22064 non-null  object
 11  filtre                              16800 non-null  ob

In [7]:
# Show the data
train[['APP_Libelle_activite_etablissement','filtre','ods_type_activite']].head(20)

,APP_Libelle_activite_etablissement,filtre,ods_type_activite
0,Restaurant,Restaurant,Autres
1,Libre service|Alimentation générale,Libre service|Alimentation générale,Autres
2,Restaurant,Restaurant,Autres
3,Purification/Expédition de coquillages,NaN,Produits de la mer et d'eau douce
4,Producteur fermier,Producteur fermier,Autres
5,Découpe de viande de boucherie,NaN,Viandes et produits carnés
6,Producteur fermier,Producteur fermier,Autres
7,Restaurant,Restaurant,Autres
8,Pêche de production primaire,NaN,Produits de la mer et d'eau douce
9,Restaurant,Restaurant,Autres


**Preprocessing**

In [8]:
train['Synthese_eval_sanit'].value_counts()

Synthese_eval_sanit
Satisfaisant                     13009
Très satisfaisant                 8226
A améliorer                       1327
A corriger de manière urgente      109
Name: count, dtype: int64

In [9]:
# Map numeric values to string labels for the target variable
sanit_map = {
    'Très satisfaisant': 1,
    'Satisfaisant': 2,
    'A améliorer': 3,
    'A corriger de manière urgente': 4
}
train['eval_sanit'] = train['Synthese_eval_sanit'].map(sanit_map)
train['eval_sanit'].value_counts()

eval_sanit
2    13009
1     8226
3     1327
4      109
Name: count, dtype: int64

In [10]:
train.rename(columns={
    'Libelle_commune': 'commune',
    'APP_Libelle_activite_etablissement': 'activity'
    }, inplace=True)

test.rename(columns={
    'Libelle_commune': 'commune',
    'APP_Libelle_activite_etablissement': 'activity'
    }, inplace=True)

In [11]:
# List of predictors
iv_vars = ['Code_postal', 'commune', 'Date_inspection', 'activity']

In [12]:
# Check missing values for each column in 'iv_vars'
missing_values = train[iv_vars].isnull().sum()

# Print total NAs across selected variables
print(f"Total NAs: {missing_values.sum()}")

# Print missing values for each variable
print(missing_values)

Total NAs: 0
Code_postal        0
commune            0
Date_inspection    0
activity           0
dtype: int64


In [13]:
def feature_engineering(df):
    # Convert 'Date_inspection' to datetime, taking into account the timezone
    df['Date_inspection'] = pd.to_datetime(df['Date_inspection'], utc=True, errors='coerce')
    
    # Extract year, month, day
    df['year'] = df['Date_inspection'].dt.year
    df['month'] = df['Date_inspection'].dt.month
    df['day'] = df['Date_inspection'].dt.day

    # Day of the week (Monday=0, Sunday=6)
    df['day_of_week'] = df['Date_inspection'].dt.dayofweek
    
    # Weekend    
    df['weekend'] = df['Date_inspection'].dt.weekday >= 5
    df['weekend'] = df['weekend'].astype(int)

    # Week of the year
    df['week_of_year'] = df['Date_inspection'].dt.isocalendar().week

    # Quarter of the year
    df['quarter'] = df['Date_inspection'].dt.quarter

    # Is Month Start/End
    df['is_month_end'] = df['Date_inspection'].dt.is_month_end.astype(int)
    
    df['department'] = df['Code_postal'].astype(str).str[:2]
    
    return df

In [14]:
train = feature_engineering(train)

test = feature_engineering(test)

In [15]:

# def combine_and_clean_activities(df, columns):
#     # Combine the activity-related columns into a single string per row for analysis
#     df['activity_combined'] = df[columns].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

#     # Clean the text: convert to lowercase and remove special characters
#     df['activity_combined'] = df['activity_combined'].str.lower().str.replace(r'[^a-z\s]', ' ', regex=True)

#     # Remove duplicate words within each row
#     df['activity_combined'] = df['activity_combined'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.index)))

#     return df['activity_combined']

# def get_top_n_words(corpus, n=None):
#     """
#     List the top n words in a vocabulary according to occurrence in a text corpus.
#     """
#     vec = CountVectorizer().fit(corpus)
#     bag_of_words = vec.transform(corpus)
#     sum_words = bag_of_words.sum(axis=0) 
#     words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
#     words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
#     return words_freq[:n]

# # Assuming 'train' and 'test' DataFrames are already defined and include the necessary columns
# activity_columns = ['activity', 'filtre', 'ods_type_activite']

# # Combine and clean the activity columns
# train['activity_combined'] = combine_and_clean_activities(train, activity_columns)
# test['activity_combined'] = combine_and_clean_activities(test, activity_columns)

# # Find the top n words from the training set
# top_words = get_top_n_words(train['activity_combined'], n=50)  # n is the number of top words to keep
# top_n_words = [word for word, freq in top_words]

# # Initialize CountVectorizer with the top n words as vocabulary
# vectorizer = CountVectorizer(vocabulary=top_n_words)

# # Fit and transform the training set
# X_train = vectorizer.fit_transform(train['activity_combined'])

# # Transform the test set using the same vectorizer
# X_test = vectorizer.transform(test['activity_combined'])

# # Create DataFrame from the encoded features
# train_encoded = pd.DataFrame(X_train.toarray(), columns=vectorizer.get_feature_names_out())
# test_encoded = pd.DataFrame(X_test.toarray(), columns=vectorizer.get_feature_names_out())

# # Concatenate with the original DataFrames
# # Drop the raw text columns and 'activity_combined' as they are now redundant
# train_preprocessed = pd.concat([train.drop(columns=activity_columns + ['activity_combined']), train_encoded], axis=1)
# test_preprocessed = pd.concat([test.drop(columns=activity_columns + ['activity_combined']), test_encoded], axis=1)


In [16]:
# Check for invalid data 
train['department'].unique()

array(['33', '93', '20', '50', '86', '90', '75', '97', '57', '94', '74',
       '56', '14', '59', '29', '37', '73', '80', '47', '15', '35', '71',
       '13', '01', '83', '89', '64', '77', '31', '72', '44', '22', '42',
       '38', '06', '39', '34', '04', '69', '60', '88', '25', '66', '61',
       '68', '41', '07', '79', '17', '08', '49', '24', '62', '54', '78',
       '76', '30', '05', '32', '12', '16', '11', '53', '95', '67', '92',
       '81', '19', '40', '85', '65', '70', '52', '63', '43', '46', '55',
       '09', '26', '51', '91', '98', '23', '84', '27', '10', '36', '58',
       '18', '03', '45', '82', '48', '87', '28', '00', '02', '21', 'AD'],
      dtype=object)

In [17]:
# # Check for overseas departments

# overseas_departments_train = train[train['Code_postal'].isin(['971', '972', '973', '974', '975', '976'])]
# overseas_departments_test = test[test['Code_postal'].isin(['971', '972', '973', '974', '975', '976'])]
# print("nb of overseas_departments in train: ", len(overseas_departments_train))
# print("nb of overseas_departments in test: ", len(overseas_departments_test))

In [18]:
# Replace 'Region' values from 'AD' to '00'
train['department'] = train['department'].replace('AD', 'missing')
train['department'] = train['department'].replace('00', 'missing')

test['department'] = test['department'].replace('00', 'missing')

In [19]:
test['department'].unique()

array(['91', '92', '74', '78', '29', '80', '38', '63', '14', '73', '84',
       '41', '10', '59', '36', '42', '54', '83', '58', '01', '97', '94',
       '34', '48', '75', '62', '66', '76', '85', '33', '18', '71', '60',
       '28', '95', '55', '49', '32', '56', '70', '88', '72', '31', '30',
       '15', '17', '20', '06', '22', '53', '37', '23', '26', '51', '61',
       '05', '25', '77', '57', '68', '79', '19', '47', '12', '43', '89',
       '16', '07', '86', '46', '65', '69', '81', '82', 'missing', '45',
       '09', '03', '0[', '40', '13', '35', '44', '98', '93', '39', '02',
       '87', '24', '11', '04', '50', '67', '08', '64', '52', '27', '21',
       '90'], dtype=object)

In [20]:
# Target variable
dv_var = 'eval_sanit'

# List of predictors
iv_vars = ['department', 'commune', 'weekend', 'activity', 'filtre',
           'ods_type_activite', 'year', 'month', 'day', 'day_of_week',
           'week_of_year','quarter', 'is_month_end']

# List of predictors to encode
to_encode = ['department', 'commune', 'activity']

# List of predictors not to encode
# Use set operations to get the items in iv_vars but not in to_encode
not_encoded = list(set(iv_vars) - set(to_encode))

In [21]:
# For the training dataset, include both independent variables (iv_vars) and the dependent variable (dv_var_name)
train = train[iv_vars + [dv_var]]

test = test[['trustii_id'] + iv_vars]

In [24]:
# Load the spaCy model for French
nlp = spacy.load('fr_core_news_sm')

# Function to lemmatize text using spaCy
def lemmatize_text(text, nlp_model):
    doc = nlp_model(text)
    return ' '.join(token.lemma_ for token in doc if not token.is_punct | token.is_space)


In [ ]:
# Function to preprocess and vectorize the activities data
def vectorize_activities(train_data, test_data, text_column, nlp_model, top_n=20):
    # Lemmatize the text data in the train set
    train_data['lemmatized'] = train_data[text_column].apply(lambda x: lemmatize_text(x, nlp_model))

    # Identify the top n-grams in the train set
    vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=top_n, token_pattern=r"(?u)\b\w\w+\b")
    X_train = vectorizer.fit_transform(train_data['lemmatized'])

    # Transform the test data using the fitted vectorizer
    test_data['lemmatized'] = test_data[text_column].apply(lambda x: lemmatize_text(x, nlp_model))
    X_test = vectorizer.transform(test_data['lemmatized'])

    # Create DataFrames from the transformed data
    train_df = pd.DataFrame(X_train.toarray(), columns=vectorizer.get_feature_names_out())
    test_df = pd.DataFrame(X_test.toarray(), columns=vectorizer.get_feature_names_out())

    return train_df, test_df

In [ ]:
# Combine and clean the activity-related text in both the train and test sets
activity_columns = ['activity', 'filtre', 'ods_type_activite']
train['activity_text'] = train[activity_columns].fillna('').agg(' '.join, axis=1).str.lower()
test['activity_text'] = test[activity_columns].fillna('').agg(' '.join, axis=1).str.lower()

# Apply the vectorization to the activity data
train_vectorized, test_vectorized = vectorize_activities(train, test, 'activity_text', nlp, top_n=20)

# Concatenate the vectorized data with the original train and test DataFrames
train_final = pd.concat([train, train_vectorized], axis=1)
test_final = pd.concat([test, test_vectorized], axis=1)

# Remove the original activity columns and the intermediate text columns to avoid redundancy
columns_to_drop = activity_columns + ['activity_text', 'lemmatized']
train_final.drop(columns=columns_to_drop, inplace=True)
test_final.drop(columns=columns_to_drop, inplace=True)

In [ ]:
# # Download necessary NLTK resources if they haven't been already
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')

# # Define a lemmatization function using NLTK
# def lemmatize_text(text):
#     lemmatizer = WordNetLemmatizer()
#     word_list = word_tokenize(text)
#     lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list if w not in stopwords.words('english')])
#     return lemmatized_output

# # Update the function to process the text data
# def process_text(df, columns):
#     # Combine the activity-related columns into a single string per row for analysis
#     df['activity_combined'] = df[columns].apply(lambda x: ' '.join(x.dropna()), axis=1)

#     # Clean the text: convert to lowercase, remove special characters and lemmatize
#     df['activity_combined'] = df['activity_combined'].str.lower().str.replace(r'[^\w\s]', ' ', regex=True)
#     df['activity_combined'] = df['activity_combined'].apply(lemmatize_text)
#     return df['activity_combined']

# # Initialize the CountVectorizer with a better token pattern and lemmatization
# vectorizer = CountVectorizer(token_pattern=r'\b\w+\b', max_features=20)

# # Assuming 'train' and 'test' are your DataFrame and they have been loaded
# train['activity_combined'] = process_text(train, ['APP_Libelle_activite_etablissement', 'filtre', 'ods_type_activite'])
# test['activity_combined'] = process_text(test, ['APP_Libelle_activite_etablissement', 'filtre', 'ods_type_activite'])

# # Fit on the training data and transform the training data
# X_train = vectorizer.fit_transform(train['activity_combined'])

# # Transform the test data using the fitted vectorizer (without refitting)
# X_test = vectorizer.transform(test['activity_combined'])

# # Convert the matrix of n-grams into DataFrames
# train_ngrams_df = pd.DataFrame(X_train.toarray(), columns=vectorizer.get_feature_names_out())
# test_ngrams_df = pd.DataFrame(X_test.toarray(), columns=vectorizer.get_feature_names_out())


In [25]:
train_final.head()

,department,commune,weekend,year,month,day,day_of_week,week_of_year,quarter,is_month_end,...,producteur fermier,produire,produit,produit carner,restauration,restauration collectif,restaurer,restaurer autre,restaurer restaurer,viande
0,33,Bègles,0,2024,2,5,0,6,1,0,...,0,0,0,0,0,0,2,1,1,0
1,93,Villemomble,0,2023,7,17,0,29,3,0,...,0,0,0,0,0,0,0,0,0,0
2,20,Lucciana,0,2023,3,8,2,10,1,0,...,0,0,0,0,0,0,2,1,1,0
3,50,Saint-Vaast-la-Hougue,0,2023,2,15,2,7,1,0,...,0,0,1,0,0,0,0,0,0,0
4,20,Partinello,0,2023,11,21,1,47,4,0,...,2,0,0,0,0,0,0,0,0,0


In [26]:
test_final.head()

,trustii_id,department,commune,weekend,year,month,day,day_of_week,week_of_year,quarter,...,producteur fermier,produire,produit,produit carner,restauration,restauration collectif,restaurer,restaurer autre,restaurer restaurer,viande
0,5883,91,BRUNOY,0,2023,3,14,1,11,1,...,0,0,0,0,0,0,0,0,0,0
1,118,92,COLOMBES,0,2023,3,21,1,12,1,...,0,0,0,0,0,0,0,0,0,0
2,3156,74,CHAMONIX MONT BLANC,0,2023,3,21,1,12,1,...,0,0,0,0,0,0,2,1,1,0
3,4577,78,LES CLAYES SOUS BOIS,0,2023,3,17,4,11,1,...,0,0,0,0,0,0,2,1,1,0
4,6998,29,PLOUGASTEL DAOULAS,0,2023,3,22,2,12,1,...,0,0,0,0,0,0,0,0,0,0


To prevent data leakage and ensure we have the same columns in both sets:
1. Fit the encoder only on the training set to determine the categories and thereby the feature names.
2. Transform both the training and test sets using this fitted encoder. This ensures that only the categories identified in the training set are encoded, maintaining consistency across datasets.
3. Handle missing columns in the test set by adding any columns that were created during training but are missing in the test set, filling them with 0s, since these categories do not appear in the test data.
4. Align the columns of the test set with the training set, ensuring they have exactly the same columns (except for the target variable and any identifiers like 'id')

We'll follow a structured approach to handle both scenarios where the test set might miss categories present in the training set or introduce new categories not seen during training. The strategy will be to ensure the model receives consistent feature sets across both training and testing phases.

In [28]:
# List of predictors to encode
to_encode = ['department', 'commune']

# Update list of predictors, excluding the to_encode variables and the target variable
iv_vars = [col for col in train_final.columns if col not in to_encode + [dv_var]]

# List of predictors not to encode
# This automatically becomes iv_vars excluding to_encode, no need for further set operations here
not_encoded = [col for col in iv_vars if col not in to_encode]

In [29]:
# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False, drop='first', dtype=int, handle_unknown='ignore')

# Fit the encoder on the training data and transform
train_encoded = encoder.fit_transform(train_final[to_encode])
train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out())
train_encoded_df.columns = train_encoded_df.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)

# Combine with non-encoded variables and the target variable
# Fixing the pd.concat() call to include DataFrames correctly
train_processed = pd.concat([train_encoded_df, train_final[not_encoded].reset_index(drop=True), train[[dv_var]].reset_index(drop=True)], axis=1)

C:\Users\mperan\.conda\envs\py\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [30]:
# Transform the test set using the fitted encoder
test_encoded = encoder.transform(test_final[to_encode])
test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out())
test_encoded_df.columns = test_encoded_df.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)

# Manually adding missing columns in the test set, filled with 0
for column in set(train_processed.columns) - set(test_encoded_df.columns) - {dv_var}:
    test_encoded_df[column] = 0

# Reorder test_encoded_df columns to match train_processed, excluding target variable
test_encoded_df = test_encoded_df.reindex(columns=(train_processed.columns.drop(dv_var)))

# Combine encoded and non-encoded variables in the test set

test_encoded_df = pd.concat([test['trustii_id'], test_encoded_df, test_final[not_encoded].reset_index(drop=True)], axis=1)
test_processed = test_encoded_df.loc[:,~test_encoded_df.columns.duplicated()]

C:\Users\mperan\.conda\envs\py\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [31]:
train_processed.head()

,department_02,department_03,department_04,department_05,department_06,department_07,department_08,department_09,department_10,department_11,...,produire,produit,produit carner,restauration,restauration collectif,restaurer,restaurer autre,restaurer restaurer,viande,eval_sanit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,1,0,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,1,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [32]:
test_processed



,trustii_id,department_02,department_03,department_04,department_05,department_06,department_07,department_08,department_09,department_10,...,producteur fermier,produire,produit,produit carner,restauration,restauration collectif,restaurer,restaurer autre,restaurer restaurer,viande
0,5883,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,118,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3156,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4577,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8834,5401,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8835,5150,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8836,5732,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8837,2999,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Train ML model

Reference:
- mlr integrated learners: https://mlr.mlr-org.com/articles/tutorial/integrated_learners.html

#### Decision Tree

In [33]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score

# Assuming dv_var_name is the column name for your target variable
dv_var_name = 'eval_sanit'  

# Prepare the features and target variable from the processed training data
X_train = train_processed.drop(columns=[dv_var_name])
y_train = train_processed[dv_var_name]

# Define the model
clf = DecisionTreeClassifier()

# Set hyperparameter tuning grid
param_grid = {
    'max_depth': [2, 5, 10],
    'min_samples_split': [10, 20]
}

# Set up cross-validation with hyperparameter tuning
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')

# Run the hyperparameter tuning with k-fold CV
grid_search.fit(X_train, y_train)

# Extract the best model
best_model = grid_search.best_estimator_

# Check cross-validation scores for the best model
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='accuracy')

print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"CV Accuracy for best model: {cv_scores.mean()}")


Best hyperparameters: {'max_depth': 10, 'min_samples_split': 20}
CV Accuracy for best model: 0.6340258023537949


In [34]:
X_test = test_processed.drop(columns=['trustii_id'])  # Drop 'id' or any column not used for predictions

# Reorder X_test columns to match the order used in X_train
X_test = X_test[train_processed.drop(columns=[dv_var_name]).columns]

# Generate predictions using the best model
predictions = best_model.predict(X_test)

# Create a DataFrame for output with 'id' and predicted scores
output_df = pd.DataFrame({'trustii_id': test_processed['trustii_id'], 'Synthese_eval_sanit': predictions})

# Output to a CSV file
output_df.to_csv('test_predictions.csv', index=False)

In [35]:
output_df

,trustii_id,Synthese_eval_sanit
0,5883,1
1,118,1
2,3156,1
3,4577,1
4,6998,1
...,...,...
8834,5401,1
8835,5150,1
8836,5732,1
8837,2999,1
